## Import and to 2D

In [61]:
import numpy
written_train =numpy.load("written_train(1).npy")
spoken_train =numpy.load("spoken_train(1).npy")
match_train = numpy.load("match_train(1).npy")
feature_amount = spoken_train[0].shape[1]
print("shape written:", written_train.shape)
print("shape spoken", spoken_train.shape)
print("shape spoken indiv:", spoken_train[4].shape)
print(feature_amount)
print(match_train.shape)


shape written: (45000, 784)
shape spoken (45000,)
shape spoken indiv: (38, 13)
13
(45000,)


In [62]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_train)):
    maxs.append(spoken_train[i].shape[0])
maxlen_spoken_train = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_train_3d= pad_sequences(spoken_train, maxlen= max(maxs))
print(spoken_train_3d.shape)

spoken_train_2d =spoken_train_3d.reshape(45000, maxlen_spoken_train*feature_amount)

spoken_train_2d 




(45000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -1,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [29]:
#print(written_train[0])
print(written_train.shape)
print(match_train.shape)

(45000, 784)
(45000,)


### Scaling/Standardizing/normalizing

In [57]:
#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
spoken_train_mmsc = mm_scaler.fit_transform(spoken_train_2d)
written_train_mmsc = mm_scaler.fit_transform(written_train)
Scaler = "MinMax"

#When using MinMax
X_values = numpy.hstack([spoken_train_mmsc, written_train_mmsc])
print(X_values.shape)
X_valSet = "X_val set for MinMax"

(45000, 1993)


In [63]:
#Standardscaler
from sklearn.preprocessing import StandardScaler

SS_scaler = StandardScaler()
spoken_train_SSscaler = SS_scaler.fit_transform(spoken_train_2d)
written_train_SSscaler = SS_scaler.fit_transform(written_train)

Scaler = "Standard scaler"

#When using standard scaler
X_values = numpy.hstack([spoken_train_SSscaler, written_train_SSscaler])
print(X_values.shape)
X_valSet = "X_val set for standard scaler"

(45000, 1993)


In [3]:
#Max Abs
from sklearn.preprocessing import MaxAbsScaler
MaxAbs_scaler = MaxAbsScaler()
spoken_train_MaxAbsSc = MaxAbs_scaler.fit_transform(spoken_train_2d)
written_train_MaxAbsSc = MaxAbs_scaler.fit_transform(written_train)
Scaler = "Max Abs"

#When using Max Abs
X_values = numpy.hstack([spoken_train_MaxAbsSc, written_train_MaxAbsSc])
print(X_values.shape)
X_valSet = "X_val set for Max Abs"

(45000, 1993)


In [9]:
#Normalizing

from sklearn.preprocessing import normalize
spoken_train_normalized = normalize(spoken_train_2d, norm='l2')
written_train_normalized = normalize(written_train, norm='l2')
Scaler = "Normalized"

#When using Normalized
X_values = numpy.hstack([spoken_train_normalized, written_train_normalized])
print(X_values.shape)
X_valSet = "X_val set for Normalizer"

(45000, 1993)


In [5]:
#When using NOTHING!!!

X_values = numpy.hstack([spoken_train_2d, written_train])
print(X_values.shape)
X_valSet = "X_val set for NOTHING"

(45000, 1993)


## Upsample of oversample

### upsample

In [32]:
#------------ Separate majority and minority classes

match_train_ = match_train.reshape((45000,1))
df_combo = numpy.hstack([X_values, match_train_])

#get row indices from True
countT = 0
rowindices_T = []
for row in match_train:
    if row == True:
        rowindices_T.append(countT)
    countT += 1
df_minority = df_combo[rowindices_T][:]    
df_minority.shape
print("Minority before upsampling:", df_minority.shape[0])


#get row indices from False
countF = 0
rowindices_F = []
for row in match_train:
    if row == False:
        rowindices_F.append(countF)
    countF += 1
df_majority = df_combo[rowindices_F][:]    

aantalmajority = df_majority.shape[0]
print("Majority before upsampling:", aantalmajority)


# ------------ Upsample minority class
from sklearn.utils import resample
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples =aantalmajority,    # to match majority class

                                 random_state=123) # reproducible results
print("na upsample:", df_minority_upsampled.shape[0])
# Combine majority class with upsampled minority class
df_upsampled = numpy.vstack([df_majority, df_minority_upsampled])
X_values_upsampled = df_upsampled[:,:-1]
match_train_upsampled = df_upsampled[:,-1]
X_train = X_values_upsampled
y_train = match_train_upsampled

Minority before upsampling: 4539
Majority before upsampling: 40461
na upsample: 40461


### Oversampling

In [64]:
from imblearn.over_sampling import SMOTE

y_train = match_train
X_train = X_values

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==0)))
print("Before OverSampling, counts of label '2': {} ".format(sum(y_train==1)))

sm = SMOTE(random_state=4)
X_res, y_res = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of X: {}'.format(X_res.shape))
print('After OverSampling, the shape of y: {} \n'.format(y_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_res==0)))
print("After OverSampling, counts of label '2': {}".format(sum(y_res==1)))


X= numpy.array(X_res)
y= numpy.array(y_res)

X_train=X
y_train=y
print(y_train.shape)

Before OverSampling, counts of label '1': 40461
Before OverSampling, counts of label '2': 4539 
After OverSampling, the shape of X: (80922, 1993)
After OverSampling, the shape of y: (80922,) 

After OverSampling, counts of label '1': 40461
After OverSampling, counts of label '2': 40461
(80922,)


## models

In [147]:
print( numpy.unique( predy ) )
print(pd.value_counts(predy))

[0. 1.]
0.0    14612
1.0    12362
dtype: int64


### Test CNN

In [28]:
from keras.models import Sequential
from keras.layers import Conv1D
model = Sequential()
model = Sequential()
model.add(Conv1D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(1993)))
model.add(MaxPooling1D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv1D(64, (5, 5), activation='relu'))
model.add(MaxPooling1D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))



TypeError: 'int' object is not iterable

In [83]:
#from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/3, random_state=999)



from keras.models import Sequential
from keras.layers import Dense, Dropout


model = Sequential()
model.add(Dense(64, input_dim=1993, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

hist = model.fit(X_train, y_train,
          epochs=20,
          batch_size=208,
          validation_split=0.2)

#score = model.evaluate(X_val, y_val, batch_size=128)
#print(model.metrics_names)
#print(score)

Train on 64737 samples, validate on 16185 samples
Epoch 1/20
64737/64737 [==============================] - 2s 37us/step - loss: 0.6190 - acc: 0.6731 - val_loss: 0.5162 - val_acc: 0.8561
Epoch 2/20
64737/64737 [==============================] - 1s 23us/step - loss: 0.4390 - acc: 0.8035 - val_loss: 0.4511 - val_acc: 0.8833
Epoch 3/20
64737/64737 [==============================] - 1s 23us/step - loss: 0.3605 - acc: 0.8521 - val_loss: 0.3319 - val_acc: 0.9396
Epoch 4/20
64737/64737 [==============================] - 1s 23us/step - loss: 0.3077 - acc: 0.8821 - val_loss: 0.2929 - val_acc: 0.9444
Epoch 5/20
64737/64737 [==============================] - 1s 23us/step - loss: 0.2702 - acc: 0.8990 - val_loss: 0.2315 - val_acc: 0.9623
Epoch 6/20
64737/64737 [==============================] - 1s 23us/step - loss: 0.2473 - acc: 0.9111 - val_loss: 0.2057 - val_acc: 0.9661
Epoch 7/20
64737/64737 [==============================] - 2s 25us/step - loss: 0.2254 - acc: 0.9226 - val_loss: 0.1663 - val_acc

## Test data

In [78]:
written_test =numpy.load("written_test(1).npy")
spoken_test =numpy.load("spoken_test(1).npy")

In [79]:
import pandas as pd
from numpy import array

maxs = []
for i in range(0, len(spoken_test)):
    maxs.append(spoken_test[i].shape[0])
maxlen_spoken_test = max(maxs)

from keras.preprocessing.sequence import pad_sequences

spoken_test_3d= pad_sequences(spoken_test, maxlen= max(maxs))
print(spoken_test_3d.shape)

spoken_test_2d =spoken_test_3d.reshape(15000, maxlen_spoken_test*feature_amount)

spoken_test_2d

(15000, 93, 13)


array([[ 0,  0,  0, ..., -1,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ..., -2,  0,  1],
       ...,
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ..., -2, -1,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

#### scalers

In [24]:
#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
spoken_test_mmsc = mm_scaler.fit_transform(spoken_test_2d)
written_test_mmsc = mm_scaler.fit_transform(written_test)
Scaler = "MinMax"

#When using MinMax
X_test = numpy.hstack([spoken_test_mmsc, written_test_mmsc])


(45000, 1993)


In [80]:
#Standardscaler
from sklearn.preprocessing import StandardScaler

SS_scaler = StandardScaler()
spoken_test_SSscaler = SS_scaler.fit_transform(spoken_test_2d)
written_test_SSscaler = SS_scaler.fit_transform(written_test)

Scaler = "Standard scaler"

#When using standard scaler
X_test = numpy.hstack([spoken_test_SSscaler, written_test_SSscaler])


In [30]:
#Max Abs
from sklearn.preprocessing import MaxAbsScaler
MaxAbs_scaler = MaxAbsScaler()
spoken_test_MaxAbsSc = MaxAbs_scaler.fit_transform(spoken_test_2d)
written_test_MaxAbsSc = MaxAbs_scaler.fit_transform(written_test)
Scaler = "Max Abs"

#When using Max Abs
X_test = numpy.hstack([spoken_test_MaxAbsSc, written_test_MaxAbsSc])


In [44]:
#Normalizing

from sklearn.preprocessing import normalize
spoken_test_normalized = normalize(spoken_test_2d, norm='l2')
written_test_normalized = normalize(written_test, norm='l2')
Scaler = "Normalized"

#When using Normalized
X_test = numpy.hstack([spoken_test_normalized, written_test_normalized])


In [5]:
#When using NOTHING!!!

X_test = numpy.hstack([spoken_test_2d, written_test])


(45000, 1993)


### Acc score

In [81]:
from sklearn.metrics import accuracy_score

y_pred = model.predict_classes(X_test, verbose=1)

print(y_pred)

print(1 in y_pred)
y_pred = y_pred.reshape(15000,)
print(y_pred)

15000/15000 [==============================] - 1s 47us/step
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
True
[0 0 0 ... 0 0 0]


In [82]:
numpy.save("attempttest9939.npy", y_pred)